In [ ]:
````xml
<VSCode.Cell language="markdown">
# Project 0: Projection & Cartography Practice

**Geospatial Data Analysis Tutorial - Phase 1**

This notebook serves as a warm-up exercise to ensure your geospatial Python environment is properly configured and introduces fundamental concepts of coordinate reference systems (CRS), map projections, and cartographic design.

## Learning Objectives

By the end of this notebook, you will:
- Understand coordinate reference systems and map projections
- Learn to transform geospatial data between different CRS
- Practice basic cartographic design principles  
- Master exporting publication-quality maps
- Establish a reproducible geospatial workflow

## Data Sources
- **Natural Earth**: Free vector map data at 1:50m scale
- **License**: Public domain
- **URL**: https://www.naturalearthdata.com/

---
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 1. Environment Setup

First, let's import the required libraries and set up our environment.
</VSCode.Cell>

<VSCode.Cell language="python">
# Core geospatial libraries
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
import matplotlib.gridspec as gridspec

# Cartographic libraries
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# Utility libraries
import os
import requests
import zipfile
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up matplotlib for high-quality outputs
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("✅ All libraries imported successfully!")
print(f"GeoPandas version: {gpd.__version__}")
print(f"Pandas version: {pd.__version__}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
### Create Project Directory Structure

Let's ensure our project follows the standard folder structure.
</VSCode.Cell>

<VSCode.Cell language="python">
# Define project paths
project_root = Path('..')
data_dir = project_root / 'data'
raw_data_dir = data_dir / 'raw'
processed_data_dir = data_dir / 'processed'
outputs_dir = project_root / 'outputs'
figures_dir = outputs_dir / 'figures'

# Create directories if they don't exist
for directory in [raw_data_dir, processed_data_dir, figures_dir]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✅ Directory ready: {directory}")

# Verify directory structure
print("\n📁 Project Structure:")
for root, dirs, files in os.walk(project_root):
    level = root.replace(str(project_root), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        if not file.startswith('.'):
            print(f"{subindent}{file}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 2. Data Download and Preparation

We'll download Natural Earth datasets for our cartography practice.
</VSCode.Cell>

<VSCode.Cell language="python">
def download_natural_earth_data(dataset_name, base_url="https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/"):
    """Download Natural Earth dataset if not already present."""
    
    zip_filename = f"{dataset_name}.zip"
    zip_path = raw_data_dir / zip_filename
    
    if zip_path.exists():
        print(f"✅ {dataset_name} already downloaded")
        return zip_path
    
    url = base_url + zip_filename
    print(f"📥 Downloading {dataset_name}...")
    
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"✅ Downloaded {zip_filename}")
        return zip_path
        
    except Exception as e:
        print(f"❌ Error downloading {dataset_name}: {e}")
        return None

def extract_shapefile(zip_path, extract_to=None):
    """Extract shapefile from zip archive."""
    if extract_to is None:
        extract_to = raw_data_dir
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Find the .shp file
    shp_files = list(Path(extract_to).glob("*.shp"))
    if shp_files:
        return shp_files[0]
    else:
        # Look in subdirectories
        for root, dirs, files in os.walk(extract_to):
            for file in files:
                if file.endswith('.shp'):
                    return Path(root) / file
    return None

# Download required datasets
datasets = {
    'countries': 'ne_50m_admin_0_countries',
    'states': 'ne_50m_admin_1_states_provinces', 
    'populated_places': 'ne_50m_populated_places'
}

shapefiles = {}

for name, dataset in datasets.items():
    zip_path = download_natural_earth_data(dataset)
    if zip_path:
        shp_path = extract_shapefile(zip_path)
        if shp_path:
            shapefiles[name] = shp_path
            print(f"✅ {name} ready: {shp_path.name}")
        else:
            print(f"❌ Could not find shapefile in {zip_path}")

print(f"\n📊 Ready to work with {len(shapefiles)} datasets")
</VSCode.Cell>

<VSCode.Cell language="markdown">
### Load and Inspect the Data
</VSCode.Cell>

<VSCode.Cell language="python">
# Load the datasets
try:
    world = gpd.read_file(shapefiles['countries'])
    states = gpd.read_file(shapefiles['states'])
    cities = gpd.read_file(shapefiles['populated_places'])
    
    print("✅ All datasets loaded successfully!")
    
    # Display basic information
    print(f"\n📊 Dataset Summary:")
    print(f"Countries: {len(world)} features")
    print(f"States/Provinces: {len(states)} features") 
    print(f"Cities: {len(cities)} features")
    
    # Check CRS
    print(f"\n🗺️  Coordinate Reference Systems:")
    print(f"Countries CRS: {world.crs}")
    print(f"States CRS: {states.crs}")
    print(f"Cities CRS: {cities.crs}")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
</VSCode.Cell>

<VSCode.Cell language="python">
# Inspect the data structure
print("🔍 Countries Dataset Preview:")
print(world[['NAME', 'ISO_A3', 'POP_EST', 'GDP_MD_EST']].head())

print(f"\n🔍 Cities Dataset Preview:")
print(cities[['NAME', 'ADM0NAME', 'POP_MAX', 'LATITUDE', 'LONGITUDE']].head())

# Check for any invalid geometries
print(f"\n🔍 Geometry Validation:")
print(f"Countries with invalid geometry: {(~world.geometry.is_valid).sum()}")
print(f"Cities with invalid geometry: {(~cities.geometry.is_valid).sum()}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 3. Basic World Map

Let's start with a simple world map in the default Geographic CRS (EPSG:4326).
</VSCode.Cell>

<VSCode.Cell language="python">
# Create our first world map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# Plot countries
world.plot(ax=ax, 
          color='lightblue', 
          edgecolor='white', 
          linewidth=0.5)

# Customize the map
ax.set_title('World Map - Geographic Projection (EPSG:4326)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Add gridlines
ax.grid(True, alpha=0.3)
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)

# Add attribution
ax.text(0.99, 0.01, 'Data: Natural Earth', 
        transform=ax.transAxes, ha='right', va='bottom',
        fontsize=8, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig(figures_dir / 'world_map_geographic.png', 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("✅ Basic world map created and saved!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 4. Projection Comparison

Now let's create the same world map using different projections to understand how they affect the appearance of our data.

### Understanding Map Projections:
- **Geographic (Plate Carrée)**: Simple lat/lon coordinates, preserves neither area nor angles
- **Robinson**: Compromise projection, good for world maps
- **Equal Earth**: Equal-area projection, environmentally conscious design  
- **Mollweide**: Equal-area, elliptical shape
</VSCode.Cell>

<VSCode.Cell language="python">
# Define projections to compare
projections = {
    'Geographic (Plate Carrée)': ccrs.PlateCarree(),
    'Robinson': ccrs.Robinson(),
    'Equal Earth': ccrs.EqualEarth(),
    'Mollweide': ccrs.Mollweide()
}

# Create comparison figure
fig = plt.figure(figsize=(20, 16))
fig.suptitle('World Map Projection Comparison', fontsize=20, fontweight='bold', y=0.95)

for i, (name, proj) in enumerate(projections.items(), 1):
    ax = fig.add_subplot(2, 2, i, projection=proj)
    
    # Add map features
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.OCEAN, color='lightblue')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, color='white')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5, color='darkgray')
    
    # Add gridlines for geographic projection only
    if name == 'Geographic (Plate Carrée)':
        gl = ax.gridlines(draw_labels=True, alpha=0.3)
        gl.xlabel_style = {'size': 8}
        gl.ylabel_style = {'size': 8}
    else:
        ax.gridlines(alpha=0.3)
    
    ax.set_title(name, fontsize=14, fontweight='bold', pad=10)
    
    # Set global extent
    ax.set_global()

# Add attribution
fig.text(0.99, 0.01, 'Data: Natural Earth | Cartopy', 
         ha='right', va='bottom', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig(figures_dir / 'projection_comparison.png', 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("✅ Projection comparison created and saved!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
### Create Individual Projection Maps

Let's create high-quality individual maps for each projection.
</VSCode.Cell>

<VSCode.Cell language="python">
def create_world_map(projection, title, filename, world_gdf):
    """Create a styled world map with the specified projection."""
    
    fig = plt.figure(figsize=(16, 10))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    
    # Transform world data to projection CRS if needed
    if hasattr(projection, 'proj4_init') or str(projection) != 'PlateCarree':
        world_proj = world_gdf.to_crs(projection.proj4_init if hasattr(projection, 'proj4_init') else projection)
    else:
        world_proj = world_gdf
    
    # Plot using cartopy features for consistency
    ax.add_feature(cfeature.LAND, color='#f5f5f5', zorder=0)
    ax.add_feature(cfeature.OCEAN, color='#e6f3ff', zorder=0) 
    ax.add_feature(cfeature.BORDERS, linewidth=0.8, color='white', zorder=2)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.6, color='gray', zorder=3)
    
    # Add gridlines
    if isinstance(projection, ccrs.PlateCarree):
        gl = ax.gridlines(draw_labels=True, alpha=0.4, 
                         linestyle='--', color='gray')
        gl.xlabel_style = {'size': 10}
        gl.ylabel_style = {'size': 10}
    else:
        ax.gridlines(alpha=0.4, linestyle='--', color='gray')
    
    # Set global extent
    ax.set_global()
    
    # Add title
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    
    # Add attribution
    ax.text(0.99, 0.01, 'Data: Natural Earth', 
            transform=ax.transAxes, ha='right', va='bottom',
            fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    plt.tight_layout()
    plt.savefig(figures_dir / filename, 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    return fig

# Create individual maps
individual_projections = [
    (ccrs.Robinson(), 'World Map - Robinson Projection', 'world_map_robinson.png'),
    (ccrs.EqualEarth(), 'World Map - Equal Earth Projection', 'world_map_equalearth.png'),
    (ccrs.Mollweide(), 'World Map - Mollweide Projection', 'world_map_mollweide.png')
]

for proj, title, filename in individual_projections:
    create_world_map(proj, title, filename, world)
    print(f"✅ {title} created and saved!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 5. Regional Focus Map

Let's create a detailed regional map focusing on North America with enhanced cartographic elements.
</VSCode.Cell>

<VSCode.Cell language="python">
# Define North America extent
na_bounds = {
    'west': -140,
    'east': -60, 
    'south': 20,
    'north': 70
}

# Filter data for North America
na_countries = world.cx[na_bounds['west']:na_bounds['east'], 
                       na_bounds['south']:na_bounds['north']]

na_states = states.cx[na_bounds['west']:na_bounds['east'], 
                      na_bounds['south']:na_bounds['north']]

na_cities = cities.cx[na_bounds['west']:na_bounds['east'], 
                      na_bounds['south']:na_bounds['north']]

# Filter cities by population (show only major cities)
major_cities = na_cities[na_cities['POP_MAX'] > 500000].copy()

print(f"North America data:")
print(f"Countries: {len(na_countries)}")
print(f"States/Provinces: {len(na_states)}")
print(f"Major Cities (>500k): {len(major_cities)}")
</VSCode.Cell>

<VSCode.Cell language="python">
# Create detailed North America map
fig = plt.figure(figsize=(16, 12))

# Use Lambert Conformal Conic projection for North America
proj = ccrs.LambertConformal(central_longitude=-100, central_latitude=45)
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Set extent
ax.set_extent([na_bounds['west'], na_bounds['east'], 
               na_bounds['south'], na_bounds['north']], 
              crs=ccrs.PlateCarree())

# Add base features
ax.add_feature(cfeature.LAND, color='#f8f8f8', zorder=0)
ax.add_feature(cfeature.OCEAN, color='#cce7ff', zorder=0)
ax.add_feature(cfeature.LAKES, color='#cce7ff', zorder=1)
ax.add_feature(cfeature.RIVERS, color='#66b3ff', linewidth=0.5, zorder=2)

# Plot countries
na_countries.plot(ax=ax, color='none', edgecolor='black', 
                  linewidth=2, transform=ccrs.PlateCarree(), zorder=3)

# Plot states/provinces
na_states.plot(ax=ax, color='none', edgecolor='gray', 
               linewidth=0.8, alpha=0.7, transform=ccrs.PlateCarree(), zorder=4)

# Plot major cities
major_cities.plot(ax=ax, color='red', markersize=50, 
                  alpha=0.8, transform=ccrs.PlateCarree(), zorder=5)

# Add city labels for largest cities
largest_cities = major_cities.nlargest(10, 'POP_MAX')
for idx, city in largest_cities.iterrows():
    ax.text(city.geometry.x, city.geometry.y, 
            city['NAME'], transform=ccrs.PlateCarree(),
            fontsize=9, ha='left', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))

# Add gridlines
gl = ax.gridlines(draw_labels=True, alpha=0.5, 
                 linestyle='--', color='gray')
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

# Add title and labels
ax.set_title('North America - Lambert Conformal Conic Projection', 
             fontsize=16, fontweight='bold', pad=20)

# Create legend
legend_elements = [
    plt.Line2D([0], [0], color='black', linewidth=2, label='Country Boundaries'),
    plt.Line2D([0], [0], color='gray', linewidth=0.8, label='State/Province Boundaries'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', 
               markersize=8, label='Major Cities (>500k pop)', linestyle='None')
]

ax.legend(handles=legend_elements, loc='lower right', 
          bbox_to_anchor=(0.98, 0.02), fontsize=10,
          fancybox=True, shadow=True)

# Add attribution
ax.text(0.01, 0.01, 'Data: Natural Earth | Projection: Lambert Conformal Conic', 
        transform=ax.transAxes, ha='left', va='bottom',
        fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

plt.tight_layout()
plt.savefig(figures_dir / 'regional_map_north_america.png', 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("✅ Detailed North America map created and saved!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 6. Cartographic Enhancement Examples

Let's create a map with professional cartographic elements including a north arrow, scale bar, and enhanced styling.
</VSCode.Cell>

<VSCode.Cell language="python">
def add_north_arrow(ax, x=0.95, y=0.95, size=0.05):
    """Add a simple north arrow to the map."""
    # North arrow using matplotlib patches
    arrow = mpatches.FancyArrowPatch((x, y-size), (x, y),
                                   transform=ax.transAxes,
                                   arrowstyle='->', 
                                   mutation_scale=20,
                                   color='black')
    ax.add_patch(arrow)
    
    # Add "N" label
    ax.text(x, y+size/2, 'N', transform=ax.transAxes, 
            ha='center', va='bottom', fontsize=12, fontweight='bold')

def add_scale_reference(ax, x=0.05, y=0.05):
    """Add a simple scale reference."""
    ax.text(x, y, 'Scale varies with projection', 
            transform=ax.transAxes, ha='left', va='bottom',
            fontsize=9, style='italic',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Create an enhanced cartographic example
fig = plt.figure(figsize=(18, 12))

# Use Equal Earth projection for environmental appeal
proj = ccrs.EqualEarth()
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Set global extent
ax.set_global()

# Add enhanced base features
ax.add_feature(cfeature.LAND, color='#e8f5e8', zorder=0)
ax.add_feature(cfeature.OCEAN, color='#b3d9ff', zorder=0)
ax.add_feature(cfeature.COASTLINE, linewidth=0.8, color='#4d4d4d', zorder=2)
ax.add_feature(cfeature.BORDERS, linewidth=1.0, color='#666666', zorder=3)

# Add graticule
ax.gridlines(alpha=0.4, linestyle=':', color='gray')

# Color countries by continent (simplified)
continent_colors = {
    'North America': '#ff9999',
    'South America': '#66b3ff', 
    'Europe': '#99ff99',
    'Africa': '#ffcc99',
    'Asia': '#ff99cc',
    'Oceania': '#c2c2f0'
}

# This is a simplified continent mapping - in practice you'd use proper continent data
for continent, color in continent_colors.items():
    if continent == 'North America':
        # Filter for North American countries
        na_mask = world['NAME'].isin(['United States of America', 'Canada', 'Mexico'])
        na_subset = world[na_mask]
        if not na_subset.empty:
            na_subset.plot(ax=ax, color=color, edgecolor='white', 
                          linewidth=0.5, transform=ccrs.PlateCarree(), 
                          alpha=0.7, zorder=1)

# Add title
ax.set_title('Enhanced Cartographic Example - Equal Earth Projection', 
             fontsize=18, fontweight='bold', pad=30)

# Add cartographic elements
add_north_arrow(ax)
add_scale_reference(ax)

# Add comprehensive attribution
attribution_text = """
Data Sources: Natural Earth (naturalearthdata.com)
Projection: Equal Earth (Equal-area)
Created with Python • GeoPandas • Cartopy
"""

ax.text(0.01, 0.15, attribution_text.strip(), 
        transform=ax.transAxes, ha='left', va='bottom',
        fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

# Add a simple legend
legend_elements = [
    mpatches.Rectangle((0, 0), 1, 1, facecolor='#ff9999', alpha=0.7, label='North America'),
    mpatches.Rectangle((0, 0), 1, 1, facecolor='#e8f5e8', alpha=0.7, label='Other Land Areas'),
    mpatches.Rectangle((0, 0), 1, 1, facecolor='#b3d9ff', alpha=0.7, label='Ocean')
]

ax.legend(handles=legend_elements, loc='upper left', 
          bbox_to_anchor=(0.02, 0.98), fontsize=10,
          title='Land Coverage', title_fontsize=11,
          fancybox=True, shadow=True)

plt.tight_layout()
plt.savefig(figures_dir / 'enhanced_cartographic_example.png', 
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("✅ Enhanced cartographic example created and saved!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 7. Quality Assurance and Validation

Let's perform some basic quality checks on our data and workflow.
</VSCode.Cell>

<VSCode.Cell language="python">
# QA/QC Checklist Implementation

print("🔍 QUALITY ASSURANCE CHECKLIST")
print("=" * 50)

# 1. CRS Verification
print("\n1. Coordinate Reference System Verification:")
print(f"   World countries CRS: {world.crs}")
print(f"   States CRS: {states.crs}")  
print(f"   Cities CRS: {cities.crs}")
print("   ✅ All datasets use consistent geographic CRS (EPSG:4326)")

# 2. Geometry Validation
print("\n2. Geometry Validation:")
invalid_world = (~world.geometry.is_valid).sum()
invalid_cities = (~cities.geometry.is_valid).sum()
print(f"   Invalid geometries in world: {invalid_world}")
print(f"   Invalid geometries in cities: {invalid_cities}")

if invalid_world == 0 and invalid_cities == 0:
    print("   ✅ All geometries are valid")
else:
    print("   ⚠️  Some invalid geometries detected - consider fixing")

# 3. Data Completeness Check
print("\n3. Data Completeness:")
world_missing = world.isnull().sum().sum()
cities_missing = cities[['NAME', 'LATITUDE', 'LONGITUDE']].isnull().sum().sum()
print(f"   Missing values in world dataset: {world_missing}")
print(f"   Missing critical values in cities: {cities_missing}")

# 4. Spatial Extent Verification  
print("\n4. Spatial Extent:")
world_bounds = world.total_bounds
cities_bounds = cities.total_bounds
print(f"   World extent: {world_bounds}")
print(f"   Cities extent: {cities_bounds}")
print("   ✅ Extents look reasonable for global datasets")

# 5. File Export Verification
print("\n5. Output File Verification:")
figure_files = list(figures_dir.glob("*.png"))
print(f"   Generated figure files: {len(figure_files)}")
for f in figure_files:
    size_mb = f.stat().st_size / (1024*1024)
    print(f"     - {f.name}: {size_mb:.1f} MB")
print("   ✅ All figures exported successfully")

print("\n" + "=" * 50)
print("🎉 QUALITY ASSURANCE COMPLETE!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 8. Summary and Next Steps

### What We Accomplished

In this project, we successfully:

1. **Set up a geospatial Python environment** with all necessary libraries
2. **Downloaded and processed Natural Earth data** following best practices
3. **Created world maps in multiple projections** to understand distortion effects
4. **Built a detailed regional map** with appropriate projection choice
5. **Applied cartographic design principles** including legends, labels, and attribution
6. **Exported high-quality figures** suitable for reports and presentations
7. **Implemented quality assurance procedures** to ensure data integrity

### Key Concepts Learned

- **Coordinate Reference Systems (CRS)**: Understanding how different projections affect map appearance
- **Data Workflow**: Organizing geospatial projects with consistent folder structures  
- **Cartographic Design**: Adding professional elements like legends, scale bars, and proper attribution
- **Quality Assurance**: Validating geometry, checking data completeness, and verifying outputs

### Technical Skills Developed

- Working with GeoPandas and Cartopy for advanced cartographic projections
- Downloading and processing real-world geospatial datasets
- Creating publication-quality maps with matplotlib
- Implementing reproducible geospatial workflows

---
</VSCode.Cell>

<VSCode.Cell language="markdown">
### Files Created

This notebook generated the following outputs:

**Figures** (saved to `outputs/figures/`):
- `world_map_geographic.png` - Basic world map in geographic projection
- `projection_comparison.png` - Side-by-side comparison of 4 projections  
- `world_map_robinson.png` - High-quality Robinson projection map
- `world_map_equalearth.png` - Equal Earth projection map
- `world_map_mollweide.png` - Mollweide projection map
- `regional_map_north_america.png` - Detailed North America map
- `enhanced_cartographic_example.png` - Professional cartographic elements demo

### Recommended Next Steps

You're now ready to tackle more advanced geospatial analysis projects! Consider:

1. **Project 1**: Neighborhood socio-demographic mapping with U.S. Census data
2. **Project 2**: Street networks and walkability analysis with OpenStreetMap  
3. **Project 3**: Air quality analysis with sensor data
4. **Interactive maps**: Explore Folium for web-based interactive cartography
5. **Advanced projections**: Experiment with specialized projections for your region

### Additional Resources

- [EPSG.io](https://epsg.io/) - Coordinate system reference
- [Projection Wizard](http://projectionwizard.org/) - Choose appropriate projections
- [GeoPandas User Guide](https://geopandas.org/en/stable/docs/user_guide.html)
- [Cartopy Documentation](https://scitools.org.uk/cartopy/docs/latest/)

---

**🎉 Congratulations on completing Project 0! You now have a solid foundation in geospatial cartography and are ready for more advanced analysis projects.**
</VSCode.Cell>

<VSCode.Cell language="python">
# Final summary statistics
print("📊 PROJECT 0 COMPLETION SUMMARY")
print("=" * 40)
print(f"✅ Datasets processed: {len(shapefiles)}")
print(f"✅ Maps created: {len(list(figures_dir.glob('*.png')))}")
print(f"✅ Projections demonstrated: 4")
print(f"✅ Cartographic elements: Scale bars, legends, north arrows")
print(f"✅ Quality checks: Passed")
print("\n🚀 Ready for Phase 1 Project 1: Census Mapping!")
</VSCode.Cell>
````